**Universidade Federal do Rio Grande do Sul**  
Departamento de Engenharia Civil (DECIV)  
ENG01007 - Análise Estrutural por Computador  
*prof. Felipe Schaedler de Almeida*

---

In [17]:
# Importando os módulos do Python necessários para esse notebook.
from math import *  # módulo com funções matemáticas

import numpy as np  # módulo para operar com arrays
import pandas as pd  # módulo para análise de dados

import AECPy.procedimentos as pmm
from AECPy.elemento import Elemento
from AECPy.material import Material
from AECPy.no import No
from AECPy.secao import SecaoRetangular

%matplotlib inline

# Exemplo: Análise de um Pórtio Espacial com cargas distribuídas pelo MRD

<img src="AEC_ex06.svg" alt="Pórtico espacial do exemplo 06" style="width:800px">



Dados do problema (forças em N e comprimentos em mm):

In [19]:
# propriedade da seção transversal
# altura
h_sec = 500  # mm
# largura
b_sec = 200  # mm

# propreidades do material
modulo_elasticidade = 200.0e3
modulo_cisalhamento = 77.0e3
coef_poisson = modulo_elasticidade / (2 * modulo_cisalhamento) - 1

# dimensões do modelo discreto (md)
x_md = 3000.0
y_md = 4000.0
z_md = 5000.0

### Defininição do caso analisado

In [20]:
caso = "c"
pz_no3 = 0.0
wz_el1 = 0.0
w3_el2 = 0.0
ang_el2 = 0.0
if caso == "a":
    pz_no3 = -10.0e3  # N
elif caso == "b":
    wz_el1 = -30.0  # N/mm
    w3_el2 = -40.0  # N/mm
elif caso == "c":
    wz_el1 = -30.0  # N/mm
    w3_el2 = -40.0  # N/mm
    ang_el2 = -60.0

print(
    "    Caso {0} \n pz no3 = {1} N \n wz el1 = {2} N/mm  \n w2 el2 = {3} N/mm \n ang el2 = {4}°".format(
        caso, pz_no3, wz_el1, w3_el2, ang_el2
    )
)

    Caso c 
 pz no3 = 0.0 N 
 wz el1 = -30.0 N/mm  
 w2 el2 = -40.0 N/mm 
 ang el2 = -60.0°


### Dados gerais da análise de pórtico espacial (P3D)


In [21]:
No.iniciar("PE")
display("Informações do nó:", No.info())

Elemento.iniciar("PE")

'Informações do nó:'

{'tipo': 'PE',
 'ndim': 3,
 'ngdl': 6,
 'eixos locais': (1, 2, 3),
 'eixos globais': ('x', 'y', 'z'),
 'gdls globais': ('ux', 'uy', 'uz', 'rx', 'ry', 'rz'),
 'gdls locais': ('u1', 'u2', 'u3', 'r1', 'r2', 'r3'),
 'forças globais': ('fx', 'fy', 'fz', 'mx', 'my', 'mz')}

## Dados do problema (em objetos):
Dados de entrada em objetos para aplicação no *procedimento geral* do método da rigidez direta

<span style="color:blue"> Os dados são armazenados em objetos definidos pelas classes `Material`, `Secao`, `No` e `Elemento` . <span>

In [22]:
# materiais:
mat1 = Material(modulo_elasticidade, coef_poisson, nome="Material do problema")

# seção transversal
sec1 = SecaoRetangular(mat1, b_sec, h_sec, nome="Seçaõ retangular")

# nós-----------------coordenadas, apoios, cargas
nos = []
nos.append(No([0, 0, 0]))
nos.append(No([0, 0, z_md]))
nos.append(No([x_md, 0, z_md]))
nos.append(No([x_md, y_md, z_md]))

nos[0].definir_apoio("todos")
nos[3].definir_carga(fz=pz_no3)

# elementos------------- conectividades, seção transversal, trechos rígidos, rotação dos eixos locais
els = []
els.append(Elemento(nos[0], nos[1], sec1))
els.append(Elemento(nos[1], nos[2], sec1))
els.append(Elemento(nos[2], nos[3], sec1))

els[2].alterar_eixos_locais = ang_el2

els[1].definir_carga(wz=wz_el1)
els[2].definir_carga(w3=w3_el2)

## Procedimento geral do Método da Rigidez Direta

### 1) Cálculos preliminares

#### 1.1) Cálculo do número total de gdl do problema

In [23]:
# Número total de gdl da estrutura
num_nos = len(nos)
num_els = len(els)
num_gdl_no = No.info("ngdl")
num_gdl_global = num_nos * num_gdl_no

#### 1.2) Definição dos gdl de cada nó:

- Os gdl são numerados segundo a ordem crescente da numeração dos nós.
- As translações na $u_{xi}$ e $u_{yi}$ e a rotação $\theta_{zi}$ são o primeiro, segundo e terceiro, respectivametne, gdl de cada nó $i$.
- Os gdl são agrupados em gdl livres (superíndice $F$) e de apoio (superíndice $S$), de forma que o vetor de deslocamentos pode ser escrito como: $$ \boldsymbol{d} = \begin{Bmatrix} \boldsymbol{d}^{F} \\ \boldsymbol{d}^{S} \end{Bmatrix}$$

e o sistema glboal de equações fica:  

$$ \begin{bmatrix} \boldsymbol{K}^{FF} & \boldsymbol{K}^{FS} \\ \boldsymbol{K}^{SF} & \boldsymbol{K}^{SS} \end{bmatrix} \begin{Bmatrix} \boldsymbol{d}^{F} \\ \boldsymbol{d}^{S} \end{Bmatrix} = \begin{Bmatrix} \boldsymbol{p}^{F} \\ \boldsymbol{p}^{S} \end{Bmatrix}$$

<br/><br/> 


In [24]:
# atribuição dos gdl dos nós

# transferindo os índices locais dos gdl restritos dos nós para a lista gdlr_no
gdlr_no = [no.il_gdlr for no in nos]

# definindo os índices globais dos gdl de cada nó
igdl_no, (num_gdl_F, num_gdl_S) = pmm.igdl_FS(num_gdl_no, gdlr_no)

# armazenando os índices globais dos gdl em cada nó
for ino in range(num_nos):
    nos[ino].igdl = igdl_no[ino]

In [25]:
# Impressão dos gdl dos nós==================================================================================
print(" Número total de gdls:", num_gdl_global)
print(" Número total de gdls livres:", num_gdl_F)
print(" Número total de gdls restritos:", num_gdl_S)
print("\n Índice global dos gdl de cada nó:")
for ino in range(num_nos):
    print(f" no{ino} :{nos[ino].igdl}")

 Número total de gdls: 24
 Número total de gdls livres: 18
 Número total de gdls restritos: 6

 Índice global dos gdl de cada nó:
 no0 :[18, 19, 20, 21, 22, 23]
 no1 :[0, 1, 2, 3, 4, 5]
 no2 :[6, 7, 8, 9, 10, 11]
 no3 :[12, 13, 14, 15, 16, 17]


### 2) Formação do sistema global de equações:

#### 2.1) Cálculo da matriz de rigidez dos elementos ($\boldsymbol{K}_e$) e montagem da matriz de rigidez global da estrutura ($\boldsymbol{K}$):

 -  cálculo da matriz de rigidez segundo as coordenadas locais.
-  transformação da matriz para os eixos globais usando a matriz de transformação $R$

- Cada termo $K_{ij}$ da matriz de rigidez global $\boldsymbol{K}$ relaciona a força externa $p_i$ na direção do gdl global $i$ ao deslocamento generalizado ($d_j$) na direção do gdl $j$.

- Os termos são obtidos pela contribuição de cada elemento ($e$) que se conecta aos gdl $i$ e $j$:
  $K_{ij} = \sum_e K_{ij}^{(e)}$ 

#### 2.2) Cálculo do vetor de forças nodais equivalentes dos elementos ($\boldsymbol{\bar{p}}^{(e)}$)

- cálculo do vetor de forças de engastamento perfeito ($\boldsymbol{\bar{q}'}^{(e)}$) para elementos com carga no interior.

- transformação de $\boldsymbol{\bar{q}'}^{(e)}$ para coordenadas globais: $\boldsymbol{\bar{q}}^{(e)} = \boldsymbol{R}^{(e)T} \boldsymbol{\bar{q}'}^{(e)}$

- Vetor de forças nodais equivalentes do elemento $\boldsymbol{\bar{p}}^{(e)} = - \boldsymbol{\bar{q}}^{(e)}$

- vetor global de forças nodais equivalentes pela contribuição de cada elemento ($e$) que se conecta aos gdl $i$: $\boldsymbol{\bar{p}}_i = \sum_e \boldsymbol{\bar{p}}_i^{(e)}$


In [26]:
# inciando a matriz de rigidez global com zeros
K_global = np.zeros((num_gdl_global, num_gdl_global))
fne = np.zeros(num_gdl_global)

for el in els:
    # matriz de rigidez do elemento em coordenadas locais
    K_el_local = el.Ke_local()

    # matriz de transformação de coordenadas global-local
    R = el.R()

    # cálculo da matriz de rigidez do elemento (no sistema global)
    # transformação da matriz de rigidez para os eixos globais
    K_el_global = pmm.transf_coord(K_el_local, R, inv=True)

    # soma a contribuição do elememto à matriz de rigidez global
    pmm.espalhar(K_el_global, K_global, el.igdl)

    # Fornas nodais equivalentes em elementos carregados
    if el.carregado:
        # Reações de engastamento perfeito (rep) em coordenadas locais
        rep_el_local = el.rep()
        # rep em coordenadas globais
        rep_el = pmm.transf_coord(rep_el_local, R, inv=True)
        # somando ao vetor de forças nodais equivalentes
        pmm.espalhar(-rep_el, fne, el.igdl)

In [27]:
# Impressão da matriz de rigidez global =============================================================
with np.printoptions(precision=0, linewidth=100, suppress=True):
    print("\n K global: \n", K_global)


 K global: 
 [[ 7.e+06  0.e+00  0.e+00  0.e+00 -1.e+08  0.e+00 -7.e+06  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00
   0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00 -4.e+04  0.e+00  0.e+00  0.e+00 -1.e+08  0.e+00]
 [ 0.e+00  4.e+04  0.e+00  2.e+07  0.e+00  4.e+07  0.e+00 -3.e+04  0.e+00  0.e+00  0.e+00  4.e+07
   0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00 -6.e+03  0.e+00  2.e+07  0.e+00  0.e+00]
 [ 0.e+00  0.e+00  4.e+06  0.e+00 -3.e+08  0.e+00  0.e+00  0.e+00 -2.e+05  0.e+00 -3.e+08  0.e+00
   0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00 -4.e+06  0.e+00  0.e+00  0.e+00]
 [ 0.e+00  2.e+07  0.e+00  8.e+10  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00 -3.e+10  0.e+00  0.e+00
   0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00 -2.e+07  0.e+00  3.e+10  0.e+00  0.e+00]
 [-1.e+08  0.e+00 -3.e+08  0.e+00  9.e+11  0.e+00  0.e+00  0.e+00  3.e+08  0.e+00  3.e+11  0.e+00
   0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  0.e+00  1.e+08  0.e+00  0.e+00  0.e+00  2.e+11  0.e+00]
 

In [28]:
# Impressão do vetor de forças nodais equivalentes
# =============================================================
with np.printoptions(precision=0, linewidth=100, suppress=True):
    print("\n fne global: \n", fne)


 fne global: 
 [        0.         0.    -45000.         0.  22500000.         0.    -40000.         0.   -114282.
 -46188022. -22500000.  26666667.    -40000.         0.    -69282.  46188022.         0. -26666667.
         0.         0.         0.         0.         0.         0.]


#### 2.2) Formação do vetor de forças externas nodais ($\boldsymbol{p}$)
 - 
    O vetor e forças nodais é formado pela soma das forças externas aplicadas nos nós ($\boldsymbol{\hat{p}}$) e as forças nodais equivalentes ($\boldsymbol{\bar{p}}$), calculadas em substiuição às forças distribuídas nos elementos: $$ \boldsymbol{p} = \boldsymbol{\hat{p}}  + \boldsymbol{\bar{p}},$$


In [29]:
P_global = np.zeros(num_gdl_global)

# adicionando as forças nodais externas ao vetor de forçãs globais
for no in nos:
    if no.carregado:
        pmm.espalhar(no.p, P_global, no.igdl)

# adicionando o vetor de forças nodais equivalentes
P_global += fne

In [30]:
# Impressão do vetor de forças externas
with np.printoptions(precision=3, linewidth=100):
    print("\n P global: \n", P_global)


 P global: 
 [ 0.000e+00  0.000e+00 -4.500e+04  0.000e+00  2.250e+07  0.000e+00 -4.000e+04  0.000e+00 -1.143e+05
 -4.619e+07 -2.250e+07  2.667e+07 -4.000e+04  0.000e+00 -6.928e+04  4.619e+07  0.000e+00 -2.667e+07
  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]


### 3) Cálculo dos deslocamentos nodais pela solução do sistema global de equações

Os deslocamentos livres $\boldsymbol{d}^{F}$ são calculados resolvendo o sistema de equações reduzido 
$$ \boldsymbol{K}^{FF} \boldsymbol{d}^{F} = \boldsymbol{p}^{F}$$

In [31]:
d = np.zeros(num_gdl_global)  # vetor de deslocamentos nodias (total)

d_F = d[:num_gdl_F]  # deslocamentos livres (aponta para o trecho de d)
P_F = P_global[:num_gdl_F]  # forças nodais nos gdl livres
K_FF = K_global[
    :num_gdl_F, :num_gdl_F
]  # coeficientes de rigides associando os gdl livres

# solução do sistema de equações
d_F[:] = np.linalg.solve(K_FF, P_F)

# obs: d_F[:] deve ser usado para apontar para as posições dos gdl livres no vetor d (que armazena os resultados)

In [32]:
# Impressão dos deslocamentos de cada nó (através de um DataFrame do 'pandas')
deslocamentos_nodais = []
for no in nos:
    deslocamentos_nodais.append(pmm.reunir(d, no.igdl))

d_no = pd.DataFrame(deslocamentos_nodais, columns=No.info("gdls globais"))
d_no.style.set_caption("Deslocamentos nodais")

ImportError: Missing optional dependency 'Jinja2'. DataFrame.style requires jinja2. Use pip or conda to install Jinja2.

### 4) Cálculo das quantidades de interesse com base nos deslocamentos nodais:

#### 4.1) Cálculo das reações de apoio nos gdl impedidos:  
- As reações de apoio são obtidas por: $$ R = \boldsymbol{K}^{SF} \boldsymbol{d}^{F}- \boldsymbol{p}^{S}$$

In [ ]:
K_SF = K_global[-num_gdl_S:, :-num_gdl_S]
P_S = P_global[-num_gdl_S:]
Reacoes = K_SF @ d_F - P_S

In [ ]:
# Impressão das reações (através de um DataFrame do 'pandas')
reacoes = dict()
forcas_nodais = No.info("forças globais")
for ino, no in enumerate(nos):
    r_no = {ino: dict()}
    if no.ngdlr > 0:
        for il, ig in zip(no.il_gdlr, no.igdlr):
            r_no[ino][forcas_nodais[il]] = Reacoes[ig - num_gdl_F]
        reacoes.update(r_no)

r_no = pd.DataFrame(reacoes)
r_no = r_no.transpose()
r_no.style.set_caption("Reações")

,fx,fy,fz,mx,my,mz
0,80000.000000,-0.000000,228564.064606,277128129.211416,-150692193.816429,-160000000.000243


#### 4.2) Cálculo dos deslocamentos e esforços nos elementos:

Os esforços nos elementos são dados pela soma dos efeitos globais (calculados em função deslocamentos nodais) e os efeitos locais (calculados a partir das forças distribuídas nos elementos).

- em cada elemento (e), os deslocamentos nodais geleralizados ($\boldsymbol{d'}^{(e)}$) em cada nó de extremidade (i,j) do elemento segundo os eixos locais por

$$
\begin{equation}
    \boldsymbol{d'}^{(e)} =  
    \begin{Bmatrix} \boldsymbol{d'}_i \\ \boldsymbol{d'}_j \end{Bmatrix} =
    \boldsymbol{R}^{(e)} \boldsymbol{d}^{(e)} =
    \begin{Bmatrix}
    \boldsymbol{R} \boldsymbol{u}_i \\
    \boldsymbol{R} \boldsymbol{\theta}_i \\
    \boldsymbol{R} \boldsymbol{u}_j \\
    \boldsymbol{R}\boldsymbol{\theta}_j \end{Bmatrix}
\end{equation}
$$

com  $\boldsymbol{u}_i = \{ u_{xi} , u_{yi} , u_{zi} \}^T$ e $\boldsymbol{\theta}_i = \{ \theta_{xi} , \theta_{yi}  , \theta_{zi} \}^T$.

<br><br>

A partir dos deslocamentos nodais segundo os eixos locais do elemento ($\boldsymbol{d'}^{(e)}$), é possível calcular os deslocamentos ($\boldsymbol{u'}_i = \{ u_{1i} , u_{2i} , u_{3i} \}^T$ e $\boldsymbol{\theta'}_i = \{ \theta_{1i} , \theta_{2i}  , \theta_{3i} \}^T$) e os esforços ($N, V_2, M_3$) em qualquer ponto de coordenada $x$ no elemento, através das seguines equações em termos da variável adimensional $\xi = \dfrac{x}{L}$:

\begin{align*}
u_2(\xi) 	&= \left( 2 \xi^3 - 3 \xi^2 + 1 \right) u_{2i} 
		+ \left(   \xi^3 - 2 \xi^2 + \xi \right) L \theta_{3i} \\
		&- \left( 2 \xi^3 - 3 \xi^2 \right) u_{2j}
		+ \left(   \xi^3 -     \xi^2 \right) L \theta_{3j}
\end{align*}

<br><br>

\begin{align*}
\theta_3(\xi) 	&= 6 \left(  \xi^2 -  \xi     \right) \dfrac{u_{2i}}{L}
		 +   \left( 3\xi^2 - 4\xi + 1 \right) \theta_{3i} \\[2pt]
		&-6  \left(  \xi^2 -  \xi     \right) \dfrac{u_{2j}}{L}
		 +   \left( 3\xi^2 - 2\xi     \right) \theta_{3j}
\end{align*}

<br><br>

$$
V_2(\xi) = - \dfrac{EI_3}{L^2} \left[ 12 \dfrac{u_{2i}}{L}  + 6 \theta_{3i}  
		   - 12 \dfrac{u_{2j}}{L}  + 6  \theta_{3j} \right]
$$

<br><br>

\begin{align*}
M_3(\xi) = \dfrac{EI_3}{L} & \left[ 6 \left(  2\xi - 1 \right) \dfrac{u_{2i}}{L} \right.
		 + 2 \left(  3\xi - 2 \right) \theta_{3i} \\[10pt]
		& \left. -  6 \left(  2\xi - 1 \right)\dfrac{u_{2j}}{L}
		 +  2 \left(  3\xi - 1 \right) \theta_{3j}  \right]
\end{align*}



- <span style="color:blue"> O efeitos local das **cargas distribuídas no elemento** são dadas por: <span>

$${u_2}(\xi)
= \dfrac{w_{2i}L^4}{24EI_3}       \left( \xi^4 - 2 \xi^3 + \xi^2\right)
+ \dfrac{\Delta w_{2}L^4}{120EI_3} \left( \xi^5 - 3 \xi^3 + 2 \xi^2 \right)$$

<br>

$${\theta_3}(\xi)
= \dfrac{w_{2i}L^3}{12EI_3}       \left( 2\xi^3 - 3 \xi^2 + \xi \right)
+ \dfrac{\Delta w_{2}L^3}{120EI_3} \left( 5\xi^4 - 9 \xi^2 + 4 \xi \right)$$

<br>

$$V_2(\xi)
= \dfrac{w_{2i}L}{2}\left( 1- 2\xi \right)
+ \dfrac{\Delta w_{2}L}{20} \left( 3 - 10 \xi^2 \right)$$


$$
M_3(\xi) = \dfrac{ w_{2i} L^2 }{12} \left( 6\xi^2 - 6\xi + 1 \right) + \dfrac{\Delta w_{2}L^2}{60} \left( 10\xi^3 - 9 \xi + 2 \right) 
$$

In [ ]:
# Calculando os resultados (esforços e deslocamentos) nos elementos

# Número de pontos (igualmente espaçados) para cálculo dos esforços e deslocamentos no elemento
npt_rel = 5
# lista para armazenar os resultados de cada elemento
resultados_el = []
for iel, el in enumerate(els):
    # coordendas dos pontos igualmente espaçados ao longo do elemento onde são calculados os resultados
    x_rel = np.linspace(0, el.L, npt_rel)
    # deslocamentos do elemento em coordenadas locais
    dl = el.dl(d)
    # cálculo dos resultados nos pontos x_rel
    resultados_el.append(el.rel(dl, x_rel))

In [ ]:
# exemplo de impressão de um reultado
quantidade = "u2"
iel = 0
ponto = 3
print(
    f"Valor de {quantidade} no ponto {ponto} de coordenada x={resultados_el[iel]['x'][ponto]} do elemento {iel}: {resultados_el[iel][quantidade][ponto]}"
)

Valor de u2 no ponto 3 de coordenada x=3750.0 do elemento 0: 4.230430770652653


In [ ]:
# imprimindo todos os resultados em um elemetno elemento (através de um 'DataFrame' do pacote 'pandas')
iel = 2
print(f"\n Elemento {iel}:")
res_iel = pd.DataFrame(resultados_el[iel])
display(res_iel.T)


 Elemento 2:


,0,1,2,3,4
x,0.000000e+00,1.000000e+03,2.000000e+03,3.000000e+03,4.000000e+03
u1,9.399125e+01,9.399125e+01,9.399125e+01,9.399125e+01,9.399125e+01
N,-7.105427e-08,-7.105427e-08,-7.105427e-08,-7.105427e-08,-7.105427e-08
r1,6.028798e-03,6.028798e-03,6.028798e-03,6.028798e-03,6.028798e-03
Mt,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
u2,-1.557083e+01,-1.612165e+01,-1.667248e+01,-1.722330e+01,-1.777412e+01
r3,-5.508239e-04,-5.508239e-04,-5.508239e-04,-5.508239e-04,-5.508239e-04
V2,-9.714451e-10,-9.714451e-10,-9.714451e-10,-9.714451e-10,-9.714451e-10
M3,-4.708767e-06,-3.695488e-06,-2.710505e-06,-1.728535e-06,-7.152557e-07
u3,-9.951936e+00,-4.815081e+01,-8.909968e+01,-1.312986e+02,-1.738474e+02


### Exemplos de emprego do módulo `gráficos` e `unidades`

In [ ]:
# importando o módulo gráficos como gr
import graficos as gr

# importando a classe Conversor do módulo unidades
from unidades import Conversor

ModuleNotFoundError: No module named 'graficos'

In [ ]:
# Criando um coversor com unidades padrão de força e comprimento iguais às adotadas na análise
uni = Conversor(up_comprimento="mm", up_forca="N")

In [ ]:
# Criando diagramas com todos os reusultados e sem conversão de unidades
fig_diagrama = gr.diagramas(resultados_el[iel])

In [ ]:
# Criando diagramas para os resultados relativos à flexão em torno do eixo local 3 do elemento e com conversão de unidades para o padrão de saída definido no módulo 'graficos'AEC_ex06.ipynb
fig_diagrama = gr.diagramas(resultados_el[1], ["u2", "V2", "M3"], conv=uni)

In [ ]:
# criando uma figura com a represtnação 3D do modelo estrutural
fig_modelo = gr.modelo_3d(nos, els)